In [42]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from IPython.display import FileLink

In [43]:
# Define Activity Labels

activities = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
activity_labels = {
    'A': 'Initial Assessment',
    'B': 'Immediate CPR',
    'C': 'Stabilization and Monitoring',
    'D': 'Doctors Assessment',
    'E': 'Further Diagnostics and Imaging',
    'F': 'Transfer to Surgery or Advanced Treatment',
    'G': 'Treatment and Monitor Conditions',
    'H': 'Transfer to the ICU',
    'I': 'Discharge'
}

In [44]:
# Define Data Attribute & PHI Logic

def generate_patient_attributes():
    # Simulate based on accident victims — abnormal ranges more likely
    heart_rate = int(np.random.normal(loc=110, scale=20))  # normal ~60–100 bpm
    oxygen_saturation = round(np.random.normal(loc=88, scale=5), 1)  # normal 95–100%

    systolic = int(np.random.normal(loc=100, scale=15))  # normal ~120
    diastolic = int(np.random.normal(loc=65, scale=10))  # normal ~80
    blood_pressure = f"{systolic}/{diastolic}"

    age = int(np.clip(np.random.normal(loc=45, scale=18), 18, 90))  # adult age range

    # Determine consciousness level based on vitals
    if oxygen_saturation < 85 or heart_rate > 140 or systolic < 80:
        consciousness = 'Unconscious'
    elif oxygen_saturation < 90 or heart_rate > 120 or systolic < 90:
        consciousness = 'Drowsy'
    elif oxygen_saturation < 93 or heart_rate > 110 or systolic < 100:
        consciousness = 'Lethargic'
    else:
        consciousness = 'Alert'

    # Random imaging result
    imaging_result = random.choice([
        "No Critical Injury Found",
        "Confirmed Critical Injury"
    ])

    return {
        'Heart_Rate': heart_rate,
        'Oxygen_Saturation': oxygen_saturation,
        'Blood_Pressure': blood_pressure,
        'Age': age,
        'Consciousness_Level': consciousness,
        'Imaging_Results': imaging_result
    }

# Function to calculate PHI based on attributes
def calculate_phi(attributes):
    hr = attributes['Heart_Rate']
    ox = attributes['Oxygen_Saturation']
    bp = attributes['Blood_Pressure']
    cons = attributes['Consciousness_Level']
    systolic = int(bp.split('/')[0])

    # Simple rule-based PHI classification
    if cons == 'Unconscious' or ox < 85 or systolic < 80:
        return 'Life-Threatening'
    elif cons == 'Drowsy' or ox < 90 or hr > 130 or systolic < 90:
        return 'Critical'
    elif cons == 'Lethargic' or hr > 110 or ox < 94:
        return 'Deteriorating'
    else:
        return 'Stable'



In [45]:
# Define Routing Logic

def generate_routing_path(phi_initial, imaging_result):
    path = [('A', phi_initial)]
    phi = phi_initial
    loop_count = 0
    reached_E = False

    # --- DP1: After A ---
    if phi in ['Stable', 'Deteriorating']:
        path.append(('D', phi))
        path.append(('E', phi))  # E directly follows D
    elif phi in ['Critical', 'Life-Threatening']:
        path.append(('B', phi))
        path.append(('C', phi))

        if phi == 'Critical':
            path.append(('E', phi))  # Go to E after C
            reached_E = True

        elif phi == 'Life-Threatening':
            max_loops = random.randint(1, 4)
            for _ in range(max_loops):
                path.append(('B', phi))
                path.append(('C', phi))
            # After final loop, overwrite PHI and proceed through C → D → E
            phi = 'Critical'
            path.append(('C', phi))  # One last C to separate from B
            path.append(('D', phi))
            path.append(('E', phi))
            reached_E = True

    # --- DP3: After E ---
    if phi in ['Life-Threatening', 'Critical', 'Deteriorating'] and imaging_result == "Confirmed Critical Injury":
        path.append(('F', phi))
    else:
        path.append(('G', phi))

    # --- DP4: After G ---
    if path[-1][0] == 'G':
        if phi in ['Life-Threatening', 'Critical', 'Deteriorating'] and imaging_result == "No Critical Injury Found":
            path.append(('H', phi))
        else:
            path.append(('I', phi))

    return path


In [46]:
# Generate 10-case event log

event_log = []
start_time = datetime(2025, 1, 1, 8, 0)

for case_num in range(1, 301):  # 10 cases
    case_start = start_time + timedelta(minutes=random.randint(0, 240))
    case_id = f"{case_start.strftime('%Y%m%d')}{case_num}"

    attrs = generate_patient_attributes()
    phi = calculate_phi(attrs)
    imaging_result = random.choice(["No Critical Injury Found", "Confirmed Critical Injury"])
    path = generate_routing_path(phi, imaging_result)

    current_time = case_start
    imaging_seen = False

    for activity, phi_event in path:
        if activity == 'E':
            imaging_seen = True

        event = {
            'Case ID': case_id,
            'Activity': activity,
            'Activity Label': activity_labels[activity],
            'time:timestamp': current_time.isoformat(),
            'Heart_Rate': attrs['Heart_Rate'],
            'Oxygen_Saturation': attrs['Oxygen_Saturation'],
            'Blood_Pressure': attrs['Blood_Pressure'],
            'Age': attrs['Age'],
            'Consciousness_Level': attrs['Consciousness_Level'],
            'PHI': phi_event,
            'Imaging_Results': imaging_result if imaging_seen else ""
        }
        event_log.append(event)
        current_time += timedelta(minutes=random.randint(5, 60))

# Convert to DataFrame and export
df_event_log = pd.DataFrame(event_log)
csv_path = "Triage_Room_Process-eventlog.csv"
df_event_log.to_csv(csv_path, index=False)

# Show download link
FileLink(csv_path)

C:\Users\eelha081\OneDrive - University of Ottawa\Jupyter_Notebooks\SLR_Process_Sim\Triage_Room_Process-eventlog.csv

In [41]:
# Group event log by case ID and print activity sequences using → separator
for case_id, group in df_event_log.groupby("Case ID"):
    sequence = " → ".join(group["Activity"].tolist())
    print(f"{case_id}: {sequence}")

202501011: A → B → C → B → C → B → C → B → C → B → C → C → D → E → F
2025010110: A → B → C → B → C → B → C → C → D → E → F
20250101100: A → D → E → G → I
20250101101: A → B → C → B → C → C → D → E → F
20250101102: A → B → C → B → C → B → C → B → C → B → C → C → D → E → F
20250101103: A → B → C → E → G → H
20250101104: A → B → C → B → C → B → C → B → C → C → D → E → G → H
20250101105: A → B → C → E → G → H
20250101106: A → D → E → G → H
20250101107: A → B → C → B → C → B → C → B → C → B → C → C → D → E → F
20250101108: A → B → C → E → F
20250101109: A → B → C → B → C → B → C → C → D → E → F
2025010111: A → B → C → E → F
20250101110: A → B → C → E → F
20250101111: A → B → C → B → C → B → C → B → C → C → D → E → F
20250101112: A → B → C → E → G → H
20250101113: A → B → C → E → G → H
20250101114: A → D → E → G → H
20250101115: A → B → C → B → C → B → C → B → C → C → D → E → G → H
20250101116: A → B → C → B → C → B → C → B → C → C → D → E → G → H
20250101117: A → B → C → E → F
20250101118: 